# transform kc, ps to initial graph

In [ ]:
import pandas as pd
import networkx as nx
import json
from pathlib import Path
from unidecode import unidecode
from networkx.drawing.nx_pydot import graphviz_layout
from matplotlib import cm

prefix = 'data/umimematikucz-system_'

kc = pd.read_csv(Path(prefix + "kc.csv", ), sep=';', header=0, index_col=0)
follow = pd.read_csv(Path(prefix + "kc_follow.csv", ), sep=';', header=0, index_col=0)
#ps = pd.read_csv(Path(prefix + "ps.csv", ), sep=';', header=0, index_col=0)

SIZE = (100, 100)
def recursively_change_shape_and_color(node, shape, color):
    nx.set_node_attributes(G, {node:shape}, name='base_shape')
    nx.set_node_attributes(G, {node:color}, name='base_color')
    for pred in G.predecessors(node):
        recursively_change_shape_and_color(pred, shape, color)

shapes = ['ellipse', 'round-triangle', 'round-rectangle', 'diamond', 'round-diamond', 'round-pentagon',
          'round-hexagon', 'round-heptagon', 'round-octagon', 'vee']
colors = ['cadetblue', 'coral', 'seagreen', 'salmon', 'tomato', 'olive', 'khaki', 'navy', 'skyblue', 'teal', 'aqua']
G = nx.DiGraph(size=SIZE)

G.add_node(0, label='"Root"', base_shape='ellipse', base_color='rgb(50, 50, 50)', selected='False')
G.add_nodes_from([(int(idx), {'label':f'"{unidecode(name)}"', 'selected':'False'}) for idx, name in zip(kc.index, kc['name'])])
G.add_edges_from([(int(idx), int(parent)) for idx, parent in zip(kc.index, kc['parent'])], type='kc', updated='False')

for idx, shape, color in zip(G.predecessors(0), shapes, colors):
    recursively_change_shape_and_color(idx, shape, color)

G.add_edges_from([(int(first), int(second)) for first, second in follow[['kc1', 'kc2']].to_numpy()], type='ps', updated='False')
pos = nx.nx_agraph.pygraphviz_layout(G, prog="dot")

# application

## layout

In [ ]:
import dash_cytoscape as cyto
import dash_bootstrap_components as dbc
import json
from dash import dcc, html

# cannot be put into separate file of klay layout stops working!

def make_layout(styles, graph_layouts):
    options = [{'label':val, 'value':val} for val in graph_layouts.keys()]
    download_panel = dbc.Row([
        dbc.Col(dcc.Upload(dbc.Button('Upload', color="info"), id='upload_graph'), width=6),
        dbc.Col(dbc.Button('Download', id='download_btn', color="info"), width=6),
    ])
    graph_scale_panel = dbc.Card(dbc.Row([
        dbc.Col(dbc.CardBody(dbc.Input(id='x_scale', type='number', placeholder='X scale'))),
        dbc.Col(dbc.CardBody(dbc.Input(id='y_scale', type='number', placeholder='Y scale'))),
    ]))
    mode_panel = dbc.RadioItems(
        id="mode_btn",
        className="btn-group",
        inputClassName="btn-check",
        labelClassName="btn btn-outline-primary",
        labelCheckedClassName="active",
        options=[
            {"label": name, "value": val} for val, name in enumerate(['Explore', 'Edit KC Edge', 'Edit PS Edge'])
        ],
        value=0
    )
    layout_panel = dbc.Select(
        id='layout_select',
        placeholder=f"Layout ({options[0]['label']})",
        options=options,
    )

    tool_panel = dbc.Card(dbc.CardBody([
        dbc.Row([
            dbc.Col(download_panel),
            dbc.Col(mode_panel),
        ]),
        html.Br(),
        dbc.Row([
            dbc.Col(layout_panel),
            dbc.Col(graph_scale_panel),
        ])
    ]))

    main_graph = dbc.Row(dbc.Col(
        html.Div(cyto.Cytoscape(
            id='cytoscape',
            elements=[],
            style=styles['cytoscape'],
            layout={'name': 'preset', 'directed': True},
            responsive=False,
            autoRefreshLayout=False,
            stylesheet=styles['stylesheet']
        ))
    ))

    meta = html.Div([
        #html.Div(id = 'last_clicked', style={'display':'none'}),
        html.Div(id='test'),
        dcc.Download(id='download'),
        dcc.Store(id='graph_elems'),
    ])

    layout = html.Div(dbc.Container([
        tool_panel,
        html.Br(),
        main_graph,
        html.Br(),
        meta,
    ]))
    return layout

## app

In [ ]:
import dash_bootstrap_components as dbc
import src.graph as g
import dash_cytoscape as cyto
from dash import dcc, Input, Output
from dash import callback_context as ctx
from jupyter_dash import JupyterDash
from dash.exceptions import PreventUpdate
from src.utils import dbc_upload

app = JupyterDash(__name__, external_stylesheets=[dbc.themes.FLATLY])

styles = json.load(open('styles.json', 'r'))

edge_types = {1:'kc', 2:'ps'}

# GV layouts: 'dot', 'fdp', 'twopi', 'circo', 'sfdp'
cyto.load_extra_layouts()
layouts = {
    'GVdot': {'call': lambda G: (nx.nx_agraph.pygraphviz_layout(G, prog='dot'), {'name': 'preset'}), 'scale_x':1, 'scale_y':1},
    'CytoKlay': {'call': lambda G: (None, {'name': 'klay'}), 'scale_x':1, 'scale_y':0.5},  # TODO figure out how to pass edge weights to engine
    'GVtwopi': {'call': lambda G: (nx.nx_agraph.pygraphviz_layout(G, prog='twopi'), {'name': 'preset'}), 'scale_x':10, 'scale_y':5},
    # TODO breadthfirst with multiple roots
    # TODO graphviz experiment with layouts and scales
}

app.layout = make_layout(styles, layouts)

## callbacks

In [ ]:
@app.callback(
    Output('graph_elems', 'data'),
    Input('upload_graph', 'contents'),
    # prevent_initial_call=True,  TODO uncomment
)
def upload_graph(upload):
    """
    TODO uncomment and update graph.json
    return dbc_upload(upload)
    """
    return g.graph_as_elements(G, pos)  # the cake is a lie


@app.callback(
    Output('download', 'data'),
    Input('cytoscape', 'elements'),
    Input('download_btn', 'n_clicks'),
    prevent_initial_call=True,
)
def download_graph(elements, _):
    elements = [
        elem for elem in elements if ('updated' not in elem['data']) or (elem['data']['updated'] != 'Removed')
    ]
    for i in range(len(elements)):
        if 'updated' in elements[i]['data']:
            elements[i]['data']['updated'] = 'False'
    if ctx.triggered_id == 'download_btn':
        return dcc.send_string(json.dumps(elements), filename='updated_graph.json')
    raise PreventUpdate


# TODO delete
@app.callback(
    Output('test', 'children'),
    Input('cytoscape', 'elements'),
)
def test_out(inp):
    return ''


def update_layout(elements, layout, x_scale, y_scale):
    if layout is None:
        layout = next(iter(layouts.keys()))
    graph = g.graph_from_elements(elements, size=SIZE)
    position, cyto_layout = layouts[layout]['call'](graph)
    return g.graph_as_elements(
        graph,
        position,
        x_scale * layouts[layout]['scale_x'] if x_scale is not None else layouts[layout]['scale_x'],
        y_scale * layouts[layout]['scale_y'] if y_scale is not None else layouts[layout]['scale_y']
    ), cyto_layout


def is_this_edge(elem, source, target, edge_type):
    return g.is_element_edge(elem) and \
        elem['data']['source'] == g.garph_index_to_elements(source) and \
        elem['data']['target'] == g.garph_index_to_elements(target) and \
        elem['data']['type'] == edge_type


def update_edge(elems, source, target, mode):
    edge_type = edge_types[mode]
    for i in range(len(elems)):
        if is_this_edge(elems[i], source, target, edge_type):
            if elems[i]['data']['updated'] == 'False':
                elems[i]['data']['updated'] = 'Removed'
                return elems
            elif elems[i]['data']['updated'] == 'Added':
                elems.pop(i)
                return elems
            elif elems[i]['data']['updated'] == 'Removed':
                elems[i]['data']['updated'] = 'False'
                return elems
    elems.append(
        g.graph_edge_as_elements(
            source,
            target,
            {'type':edge_type, 'updated': 'Added'}
        )
    )
    return elems

@app.callback(
        [
            Output('cytoscape', 'elements'),
            Output('cytoscape', 'layout'),
        ],
        Input('mode_btn', 'value'),
        Input('cytoscape', 'selectedNodeData'),
        Input('cytoscape', 'elements'),
        Input('graph_elems', 'data'),
        Input('layout_select', 'value'),
        Input('cytoscape', 'layout'),
        Input('x_scale', 'value'),
        Input('y_scale', 'value'),
        prevent_initial_call=True,
)
def update_graph(mode, selected, elements, new_elements, layout, prev_layout, x_scale, y_scale):
    if ctx.triggered_id == 'graph_elems':
        return update_layout(new_elements, layout, x_scale, y_scale)
    elif ctx.triggered_id == 'layout_select' or ctx.triggered_id == 'x_scale' or ctx.triggered_id == 'y_scale':
        return update_layout(elements, layout, x_scale, y_scale)
    elif ctx.triggered[0]['prop_id'].split('.')[1] == 'selectedNodeData':
        # highlight
        graph = g.graph_from_elements(elements, size=SIZE)
        for i in range(len(elements)):
            if g.is_element_edge(elements[i]):
                continue
            elif any(elements[i]['data']['id'] == s['id'] for s in selected):
                elements[i]['data']['selected'] = 'True'
            elif any(g.graph_index_from_elements(elements[i]) in graph.successors(g.graph_index_from_elements(s)) for s in selected):
                elements[i]['data']['selected'] = 'Parent'
            elif any(g.graph_index_from_elements(elements[i]) in graph.predecessors(g.graph_index_from_elements(s)) for s in selected):
                elements[i]['data']['selected'] = 'Child'
            else:
                elements[i]['data']['selected'] = 'False'

        # add edge
        if mode >= 1 and len(selected) >= 2:
            elements = update_edge(
                elements,
                g.graph_index_from_elements(selected[-2]),
                g.graph_index_from_elements(selected[-1]),
                mode
            )
        return elements, prev_layout

    raise PreventUpdate


## run

In [ ]:
if __name__ == "__main__":
    app.run_server(debug=True)